## Clean up ideas
* Colors should be 255 based?
* Would be nice to automatically normalize vectors.
* Are there any issues with publishing the teapot wavefront?
* Constant arrays in the cuda code?

In [ ]:
import pycuda.autoinit
import numpy as np
from pycuda import gpuarray
import pycuda.driver as drv
from pycuda.compiler import SourceModule
import cv2
import itertools
import pywavefront

In [ ]:
# Compile cuda code
module = SourceModule(open("kernels_raytracer.cu", "r").read())
kernel_draw_scene = module.get_function("draw_scene")

In [ ]:
def stack_padding(it):

    def resize(row, size):
        increase = size-row.shape[1]
        
        if increase > 0:
            new = np.zeros((row.shape[0], size))
            new[:, :-increase] = row
            return new
        else:
            return row

    # Remove empty lists
    it_full = [x for x in it if len(x)>0]
        
    row_length = max([x.shape[1] for x in it_full])
    return np.concatenate( [resize(row, row_length) for row in it_full] ).astype(np.float32)

In [ ]:
def load_obj(filename, color, scale, center):
    
    scene = pywavefront.Wavefront("teapot.obj", collect_faces=True)
    color = np.array([0.6, 0.6, 0.6], dtype=np.float32)
    
    center_np = np.array(center + center + center, dtype=np.float32)
    scale_np = np.array(scale + scale + scale, dtype=np.float32)
    
    triangles = []
    for face in scene.mesh_list[0].faces:
        vertices = np.array([scene.vertices[i] for i in face], dtype=np.float32).reshape(9,)
        triangles.append(np.concatenate([color, scale_np*vertices+center_np]))
    triangles = np.array(triangles, dtype=np.float32)
    
    return triangles

In [ ]:
# Define spheres
# Format is r,g,b,x,y,z,radius

#spheres_params = np.array([[1, 0, 0, 10, 10, 20, 10]], dtype=np.float32)
#spheres_params = np.array([[1, 0, 0, 100, 100, 100, 50],
#                           [0, 1, 0, 120, 100, 100, 50]], dtype=np.float32)

#spheres_params = np.array([], dtype=np.float32)

# Irina's version
spheres_params = np.array([[1, 0, 0, 200, 200, 100, 50],
                           [0, 0.5, 1, 200, 30, 300, 50],
                           [0.5, 0, 0, 400, 350, -60, 20],
                           [1, 1, 1, 350, 350, 800, 20],
                           [0, 1, 0, 250, 220, 100, 30]], dtype=np.float32)

# Original plan
#spheres_params = np.array([[1, 0, 0, 100, 300, 100, 50],
#                           [0, 0, 1, 200, 30, 300, 50],
#                           [0.5, 0, 0, 400, 350, -60, 20],
#                           [1, 1, 1, 350, 350, 800, 20],
#                           [0, 1, 0, 150, 320, 100, 30]], dtype=np.float32)
n_spheres = len(spheres_params)

In [ ]:
# Define planes
# Format is r,g,b,x,y,z,nx,ny,nz

#planes_params = np.array([], np.float32)

# Irina's version
planes_params = np.array([[0.8, 0.8, 0.0, 0, 600, 0, 0, 0.97, 0.243],
                          [0.4, 0.9, 0.7, 0, 0, 900, 0, 0, 1.0]], dtype=np.float32)

# Original plan
#planes_params = np.array([[1.0, 1.0, 0.0, 0, 600, 0, 0, 0.97, 0.243],
#                          [0.4, 0.1, 0.6, 0, 0, 900, 0, 0, 1.0]], dtype=np.float32)

n_planes = len(planes_params)

In [ ]:
# Define triangles
# Format is r, g, b, x1, y1, z1, x2, y2, z2, x3, y3, z3

triangle_params = load_obj("teapot.obj",
                           color=[0.6, 0.6, 0.6],
                           scale=[30, -30, 30],
                           center=[150,350,500])

# Original plan
#triangle_params = np.array([[1.0, 0.5, 0.0, 250, 270, 300, 
#                             320, 300, 290, 300, 340, 295],
#                            [1.0, 0.2, 1.0, 150, 400, 350, 
#                             50, 450, 400, 110, 800, 350]], dtype=np.float32)


n_triangles = len(triangle_params)

In [ ]:
# Define the light

#light = np.array([-20, 10, 20], dtype=np.float32)

# Original plan
light = np.array([-50, 20, -100], dtype=np.float32)

In [ ]:
# Combine object types
#object_params = stack_padding([spheres_params])
object_params = stack_padding([spheres_params, planes_params, triangle_params])
object_params_gpu = gpuarray.to_gpu(object_params)

#num_objs = np.array([n_spheres], dtype=np.int32)
num_objs = np.array([n_spheres, n_planes, n_triangles], dtype=np.int32)
num_objs_gpu = gpuarray.to_gpu(num_objs)

print(num_objs)

n_types = len(num_objs)
n_params = object_params.shape[1]

light_gpu = gpuarray.to_gpu(light)

In [ ]:
# Define image
dim = 512
#dim = 8
bitmap = np.zeros((dim, dim, 3), dtype=np.int32)
bitmap_gpu = gpuarray.to_gpu(bitmap)

In [ ]:
# Calculate the bitmap
grid_size = 16
#grid_size = 1
block = (dim//grid_size, dim//grid_size, 1)
grid = (grid_size, grid_size)

print(block)
print(grid)

kernel_draw_scene(bitmap_gpu, 
                  np.int32(dim), 
                  object_params_gpu,
                  num_objs_gpu,
                  np.int32(n_params),
                  np.int32(n_types),
                  light_gpu,
                  block=block,
                  grid=grid)

In [ ]:
image = cv2.cvtColor(bitmap_gpu.get().astype(np.uint8), cv2.COLOR_RGB2BGR)
cv2.imwrite('color_img.jpg', image)